In [16]:
# import gdown
import re
import pandas as pd
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, hamming_loss, recall_score, precision_score
from tqdm import tqdm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.svm import LinearSVC

In [ ]:
# Скачиваем паркеты с диска через gdown
file_id = "1omW5pL9XnHGVpJ8Ak_He7HWKgwY6GNid"
output = "data_for_training.parquet"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

file_id = "14BCnhYugyV8dR1dopaER72-14nlINSF_"
output = "habr_articles_parsed_final.parquet"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1omW5pL9XnHGVpJ8Ak_He7HWKgwY6GNid
From (redirected): https://drive.google.com/uc?id=1omW5pL9XnHGVpJ8Ak_He7HWKgwY6GNid&confirm=t&uuid=922f53ff-3829-46e6-8786-486a53bd6853
To: /content/data_for_training.parquet
100%|██████████| 2.02G/2.02G [00:22<00:00, 88.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=14BCnhYugyV8dR1dopaER72-14nlINSF_
From (redirected): https://drive.google.com/uc?id=14BCnhYugyV8dR1dopaER72-14nlINSF_&confirm=t&uuid=d116ca1b-8821-48b6-a3ed-10b345fc3547
To: /content/habr_articles_parsed_final.parquet
100%|██████████| 877M/877M [00:11<00:00, 74.2MB/s]


'habr_articles_parsed_final.parquet'

In [3]:
articles_ddf = dd.read_parquet("/kaggle/input/pu-parquet/df_l_fin_0.parquet")
tags_ddf = dd.read_parquet("/kaggle/input/data-for-training-parquet/data_for_training.parquet", columns=["URL", "tags_classes_kmeans"])

In [4]:
# смотреть и обучать буду только на 10% данных. У меня все падает на 12gb ram в колабе.
# беру из Гошиного датасета 10% статей с кластеризованными тегами
sample_tags_ddf = tags_ddf.sample(frac=0.1, random_state=42)

In [5]:
merged_ddf = dd.merge(articles_ddf, sample_tags_ddf, left_on="url", right_on="URL", how="inner")

In [6]:
df = merged_ddf.compute()

In [7]:
cols = ['author', 'hubs', 'url', 'title_tokens', 'text_tokens', 'tags_classes_kmeans']

In [8]:
df = df[cols]
df.head(1)

,author,hubs,url,title_tokens,text_tokens,tags_classes_kmeans
0,jasiejames,GTD,https://habr.com/ru/articles/22944/,['голова' 'десять'],['метод' 'выработка' 'коллективный' 'решение' ...,"ошибки, конференция, нейросети"


In [9]:
# обрабатываю колонки
df['text_tokens'] = df['text_tokens'].apply(lambda x: re.sub(r"[^\w\s]", "", x).strip())
df['hubs'] = df['hubs'].apply(lambda x: x.split(', '))
df['tags'] = df['tags_classes_kmeans'].apply(lambda x: x.split(', '))
df.head()

,author,hubs,url,title_tokens,text_tokens,tags_classes_kmeans,tags
0,jasiejames,[GTD],https://habr.com/ru/articles/22944/,['голова' 'десять'],метод выработка коллективный решение постановк...,"ошибки, конференция, нейросети","[ошибки, конференция, нейросети]"
1,Jenek,[GTD],https://habr.com/ru/articles/31304/,['постоянно' 'совершенствовать' 'навык'],думать гений дело правый достигнуть больший со...,"машинное обучение, автоматизация","[машинное обучение, автоматизация]"
2,qiwi_k,"[Go, Компиляторы]",https://habr.com/ru/articles/846818/,['ускорить' 'программа' 'переписать' 'строчка'...,привет хабр кирилл результат новый встреча,"c++, оптимизация, ios, гаджеты","[c++, оптимизация, ios, гаджеты]"
3,mo0Oonnn,"[Тестирование IT-систем, Go, Тестирование веб-...",https://habr.com/ru/articles/836664/,['написание' 'функциональный' 'тестирование' '...,знать писать функциональный сложный юнит тест,"гаджеты, ios, тестирование","[гаджеты, ios, тестирование]"
4,SSul,"[Блог компании SimbirSoft, Программирование, I...",https://habr.com/ru/companies/simbirsoft/artic...,['оптимизация' 'go' 'повысить' 'скорость' 'эфф...,привет хабр звать itмероприятий практикум инт...,"frontend, гаджеты, архитектура, raspberry pi, ios","[frontend, гаджеты, архитектура, raspberry pi,..."


In [ ]:
df = df.drop(columns=['tags_classes_kmeans'])

In [12]:
# создаем векторы для хабов и тегов
mlb_hubs = MultiLabelBinarizer()
y_hubs = mlb_hubs.fit_transform(df['hubs'])

mlb_tags = MultiLabelBinarizer()
y_tags = mlb_tags.fit_transform(df['tags'])

# список лейблов
label_tags = mlb_tags.classes_
label_hubs = mlb_hubs.classes_

In [13]:
y_hubs.shape, y_tags.shape

((28480, 1439), (28480, 200))

In [22]:
df['tags'].apply(lambda x: len(x)).value_counts()

tags
3     6824
4     5602
2     4539
5     3813
6     2324
1     2056
7     1354
8      887
9      493
10     263
11     120
12      74
13      41
14      28
17      14
16      12
15      11
19       4
20       3
21       3
18       3
27       2
34       2
25       1
24       1
22       1
26       1
30       1
42       1
38       1
23       1
Name: count, dtype: int64

В итоге 1439 уникальных меток хабов и 200 уникальных меток тегов

In [14]:
X_train, X_test, y_train_tags, y_test_tags = train_test_split(df['text_tokens'], y_tags, test_size=0.2, random_state=42)

In [25]:
pipeline_tags = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='saga'), verbose=True, n_jobs=-1))])
pipeline_tags.fit(X_train, y_train_tags)
y_pred_tags = pipeline_tags.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  4.7min finished


In [26]:
report = classification_report(y_test_tags, y_pred_tags, output_dict=True)
hamming = hamming_loss(y_test_tags, y_pred_tags)

summary_metrics = {
    "micro avg": report["micro avg"],
    "macro avg": report["macro avg"],
    "weighted avg": report["weighted avg"]}

for metric_name, values in summary_metrics.items():
    print(f"{metric_name}:")
    print(f"  Precision: {values['precision']:.2f}")
    print(f"  Recall: {values['recall']:.2f}")
    print(f"  F1-Score: {values['f1-score']:.2f}")
print(f"Hamming Loss: {hamming:.4f}")

micro avg:
  Precision: 0.76
  Recall: 0.06
  F1-Score: 0.11
macro avg:
  Precision: 0.41
  Recall: 0.04
  F1-Score: 0.07
weighted avg:
  Precision: 0.51
  Recall: 0.06
  F1-Score: 0.09
Hamming Loss: 0.0194


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Выводы по logreg**:

Micro avg Recall: 0.06. Модель находит только 6% от всех реальных тегов

Macro avg Recall: 0.04 
Плохо предсказывает и на редких тегах
Вывод: модель пропускает многие правильные теги, то есть "не уверена" 

Относительно высокий Precision (точность)
Micro avg Precision: 0.76
Из всех предсказанных тегов 76% оказались правильными.

Macro avg Precision: 0.41
Модель точна в своих предсказаниях, но осторожничает и не предсказывает метки, в которых не уверена.

Низкий F1-Score
Micro avg F1-Score: 0.11
Баланс между Precision и Recall страдает из-за крайне низкого Recall.

Macro avg F1-Score: 0.09

Hamming Loss показывает долю неправильно предсказанных тегов для всех статей и всех возможных меток.
Значение 0.0194 означает, что модель ошибается всего на 1.9% тегов, что кажется хорошим результатом.
Но низкий Hamming Loss может достигаться, когда модель не предсказывает ничего или предсказывает очень мало меток.

Поэтому посмотрим, что там напредсказывалось.

In [27]:
def binary_to_tags(binary_matrix, label_names):
    return [", ".join([label_names[i] for i, val in enumerate(row) if val == 1]) for row in binary_matrix]

real_tags = binary_to_tags(y_test_tags, label_tags)
predicted_tags = binary_to_tags(y_pred_tags, label_tags)

df_results = pd.DataFrame({
    'real_tags': real_tags,
    'predicted_tags': predicted_tags
})

In [28]:
df_results.tail(10)

,real_tags,predicted_tags
5686,"java, javascript, python, юмор",
5687,"frontend, будущее, графика, космос, митап, фин...",космос
5688,"kubernetes, пиратство, продуктивность, теория ...",
5689,"Red Hat, bitcoin, маркетинг, пиратство, хабрахабр",
5690,"c, spring boot, МКС, игры",игры
5691,"apple, google, документация",
5692,"методология, программирование, разработка, юмор",
5693,"c, архитектура, технологии",
5694,"apple, разработка",
5695,"ruvds_статьи, unity, Стив Джобс, наука, хабрах...",


In [29]:
df_results.shape

(5696, 2)

In [30]:
print("Модель не предсказала ни одного тега для", df_results['predicted_tags'].apply(lambda x: len(x) == 0).sum(), "статей")
print("Модель предсказала больше 1ого тега для", df_results['predicted_tags'].apply(lambda x: len(x) > 1).sum(), "статей")

Модель не предсказала ни одного тега для 4279 статей
Модель предсказала больше 1ого тега для 1412 статей


из 5696 статей в тесте не предсказала ничего для 4279. Что-то все плохо.

**Linear SVC**

In [17]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 1))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

svc_clf = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svc_clf.fit(X_train_tfidf, y_train_tags)

y_pred_svc = svc_clf.predict(X_test_tfidf)

In [19]:
report = classification_report(y_test_tags, y_pred_svc, output_dict=True)
hamming = hamming_loss(y_test_tags,  y_pred_svc)

summary_metrics = {
    "micro avg": report["micro avg"],
    "macro avg": report["macro avg"],
    "weighted avg": report["weighted avg"]}

for metric_name, values in summary_metrics.items():
    print(f"{metric_name}:")
    print(f"  Precision: {values['precision']:.2f}")
    print(f"  Recall: {values['recall']:.2f}")
    print(f"  F1-Score: {values['f1-score']:.2f}")
print(f"Hamming Loss: {hamming:.4f}")

micro avg:
  Precision: 0.65
  Recall: 0.14
  F1-Score: 0.23
macro avg:
  Precision: 0.65
  Recall: 0.13
  F1-Score: 0.21
weighted avg:
  Precision: 0.61
  Recall: 0.14
  F1-Score: 0.21
Hamming Loss: 0.0188


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Метрики выросли по сравнению с логрегом

In [21]:
def binary_to_tags(binary_matrix, label_names):
    return [", ".join([label_names[i] for i, val in enumerate(row) if val == 1]) for row in binary_matrix]

real_tags = binary_to_tags(y_test_tags, label_tags)
predicted_tags = binary_to_tags(y_pred_svc, label_tags)

df_results = pd.DataFrame({
    'real_tags': real_tags,
    'predicted_tags': predicted_tags
})

In [22]:
df_results.tail(10)

,real_tags,predicted_tags
5686,"java, javascript, python, юмор",ruvds_статьи
5687,"frontend, будущее, графика, космос, митап, фин...",космос
5688,"kubernetes, пиратство, продуктивность, теория ...",
5689,"Red Hat, bitcoin, маркетинг, пиратство, хабрахабр",
5690,"c, spring boot, МКС, игры",игры
5691,"apple, google, документация",
5692,"методология, программирование, разработка, юмор","c, разработка, юмор"
5693,"c, архитектура, технологии","IoT, ios, архитектура"
5694,"apple, разработка","ios, мобильные приложения, разработка"
5695,"ruvds_статьи, unity, Стив Джобс, наука, хабрах...",bitcoin


In [23]:
print("Модель не предсказала ни одного тега для", df_results['predicted_tags'].apply(lambda x: len(x) == 0).sum(), "статей")
print("Модель предсказала больше 1ого тега для", df_results['predicted_tags'].apply(lambda x: len(x) > 1).sum(), "статей")

Модель не предсказала ни одного тега для 2417 статей
Модель предсказала больше 1ого тега для 3241 статей


Лучше стало. Она не предсказывает ничего для меньшего кол-ва статей.

**Naive Bayes**

In [24]:
nb_clf = OneVsRestClassifier(MultinomialNB(), n_jobs=-1)
nb_clf.fit(X_train_tfidf, y_train_tags)

y_pred_nb = nb_clf.predict(X_test_tfidf)

In [25]:
report = classification_report(y_test_tags, y_pred_nb , output_dict=True)
hamming = hamming_loss(y_test_tags, y_pred_nb)

summary_metrics = {
    "micro avg": report["micro avg"],
    "macro avg": report["macro avg"],
    "weighted avg": report["weighted avg"]}

for metric_name, values in summary_metrics.items():
    print(f"{metric_name}:")
    print(f"  Precision: {values['precision']:.2f}")
    print(f"  Recall: {values['recall']:.2f}")
    print(f"  F1-Score: {values['f1-score']:.2f}")
print(f"Hamming Loss: {hamming:.4f}")

micro avg:
  Precision: 0.56
  Recall: 0.02
  F1-Score: 0.04
macro avg:
  Precision: 0.20
  Recall: 0.02
  F1-Score: 0.03
weighted avg:
  Precision: 0.29
  Recall: 0.02
  F1-Score: 0.04
Hamming Loss: 0.0201


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Хуже всего по всем метрикам относительно логрег и линейного svc.

Итог: без подбора гиперпараметров лучше всего работает линейный svc